# Inputs
Get the files from Giuliano located here :

```
/afs/cern.ch/user/g/ggustavi/public/sam/test/
```

```
scp -r meehan@lxplus.cern.ch:/afs/cern.ch/user/g/ggustavi/public/sam/test .
```

# Some Modifications

[1] go into the directory and change the directory naming


In [16]:
!cd test && mv xml config

[2] Change a single line in the top level config file
```
test/config/ShapeFit_std_blind_SRCRFit_test/BkgOnly_L138p97ifb.xml
```

In [17]:
!sed '14d' test/config/ShapeFit_std_blind_SRCRFit_test/BkgOnly_L138p97ifb.xml > temp.xml 

But be careful, this command above is a hack and "irreversibly modifies" the xml config file.  If you run it twice, then it will crash everything when you try to run the xml2json command below.

In [18]:
!cp temp.xml test/config/ShapeFit_std_blind_SRCRFit_test/BkgOnly_L138p97ifb.xml

# pyhf Stuff

Start by importing the necessary tools

In [19]:
import logging
import json
import pyhf
from pyhf import Model,Workspace

Convert the xml workspace from Giuliano to a pyhf compatible JSON spec

In [20]:
!ls `pwd`/test/config/ShapeFit_std_blind_SRCRFit_test/BkgOnly_L138p97ifb.xml

/Users/meehan/work/MonoJetFit/test/config/ShapeFit_std_blind_SRCRFit_test/BkgOnly_L138p97ifb.xml


In [21]:
!pyhf xml2json --hide-progress `pwd`/test/config/ShapeFit_std_blind_SRCRFit_test/BkgOnly_L138p97ifb.xml --basedir `pwd`/test | tee monojet.json

{
    "channels": [
        {
            "name": "SR_200_cuts",
            "samples": [
                {
                    "data": [
                        3513.0
                    ],
                    "modifiers": [
                        {
                            "data": {
                                "hi_data": [
                                    7026.0
                                ],
                                "lo_data": [
                                    0.0
                                ]
                            },
                            "name": "NCB_Sys",
                            "type": "histosys"
                        }
                    ],
                    "name": "NCB_200"
                },
                {
                    "data": [
                        18604.69921875
                    ],
                    "modifiers": [
                        {
                            "data": {
                           

Validate that the spec you just created is valid

In [22]:
import json, requests, jsonschema
workspace = json.load(open('monojet.json'))
schema = requests.get('https://diana-hep.org/pyhf/schemas/1.0.0/workspace.json').json()
jsonschema.validate(instance=workspace, schema=schema)

Or by using the stuff from pyhf (because Giordon says its better in a magic way)

In [23]:
pyhf.utils.validate(workspace, 'workspace.json')

Now we need to create a "patch" because the POI for the measurement that we will perform in ouir workspace is called "mu_SIG" but this is actually not used in the bkg-only fit, so we need to remove it from the model.:
- https://python-json-patch.readthedocs.io/en/latest/tutorial.html
- http://jsonpatch.com/    

You could go and modify the initial XML file to replace "mu_SIG" by "lumi" but that is a bick of a hack.

In [24]:
import jsonpatch
patch = jsonpatch.JsonPatch([{'op': 'replace', 'path': '/measurements/0/config/poi', 'value': 'lumi'}])

Does this patch do what we think it should do?

In [25]:
sam = { "measurements": [{"config":{"poi":"CHANGETHIS"}}, {"this":2}] }
print(sam)

{'measurements': [{'config': {'poi': 'CHANGETHIS'}}, {'this': 2}]}


In [26]:
patch.apply(sam)

{'measurements': [{'config': {'poi': 'lumi'}}, {'this': 2}]}

Yeah, that looks like it replaced the thing that we wanted to change.

In [27]:
ws = pyhf.Workspace(patch.apply(workspace))

In [28]:
ws.spec

{'channels': [{'name': 'SR_200_cuts',
   'samples': [{'data': [3513.0],
     'modifiers': [{'data': {'hi_data': [7026.0], 'lo_data': [0.0]},
       'name': 'NCB_Sys',
       'type': 'histosys'}],
     'name': 'NCB_200'},
    {'data': [18604.69921875],
     'modifiers': [{'data': {'hi_data': [37209.3984375], 'lo_data': [0.0]},
       'name': 'multijet_Sys',
       'type': 'histosys'}],
     'name': 'multijet_200'},
    {'data': [892620.9375],
     'modifiers': [{'data': None, 'name': 'lumi', 'type': 'lumi'},
      {'data': {'hi_data': [933739.5625], 'lo_data': [851502.3125]},
       'name': 'ckkw_Sys',
       'type': 'histosys'},
      {'data': {'hi_data': [907795.5], 'lo_data': [877446.375]},
       'name': 'lumiSys',
       'type': 'histosys'},
      {'data': {'hi_data': [910481.125], 'lo_data': [874351.75]},
       'name': 'vjets_d1K_NNLO',
       'type': 'histosys'},
      {'data': {'hi_data': [877133.625], 'lo_data': [907836.0625]},
       'name': 'vjets_d2K_NNLO',
       'type': '

Now extract the model from the workspace

In [29]:
m = ws.model()

Print the "data" associated with the model.  Note that the "data" here has two components :
- The observed event counts in a given bin
- The NP's that will eventually be constrained.  These are referred to as "data" in the pyhf and HF lingo because they are indeed measurements that come from the data (e.g. luminosity, JES).

This is a bit of a subtle point and confusing but essential to allow us to get at the technical bits and pieces we are interested in examining.

In [30]:
d = ws.data(m)
print(d)

[105675.6875, 296032.5, 648237.4375, 61370.4921875, 89189.890625, 1581806.75, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Display the initial values of the nuissance parameters in our model.

In [31]:
m.config.suggested_init()

[0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0]

Evaluate the value of the likelihood of the model and the data at the value of the NP's that are their initial starting values.

In [32]:
m.logpdf(m.config.suggested_init(), d)

array([-52.34599713])

You can toggle the available backends here, which is the machinery that will be used to perform the likelihood minimization.

This call will let you know what backends you have at your disposal, and then you can perform the minimization below by choosing from among the entries in this list.

In [33]:
pyhf.get_backend()

(<pyhf.tensor.numpy_backend.numpy_backend at 0x1103a8f90>,
 <pyhf.optimize.opt_scipy.scipy_optimizer at 0x110395590>)

We will choose the second one, which on Sam's machine is scipy_optimizer

In [34]:
opt = pyhf.get_backend()[1]

Using this optimizer we will now perform the unconstrained fit, meaning that all NP's and the POI are free to be pulled.

This will return the values of the NP's that minimize the likelihood

In [35]:
bfit = opt.unconstrained_bestfit(pyhf.utils.loglambdav, d, m, m.config.suggested_init(), m.config.suggested_bounds() )

We can now print the values of those NP's.  Note that since the input data is the asimov dataset, they are very close to the input values

In [36]:
bfit

array([ 5.94668452e-08,  0.00000000e+00,  9.99999990e-01, -3.34501004e-08,
       -1.67250502e-08, -1.48667113e-08, -5.57501673e-09,  5.57501673e-08,
        3.34501004e-08, -3.15917615e-08,  5.76085062e-08,  1.00000004e+00,
       -2.41584058e-08, -5.57501673e-09, -1.67250502e-08,  1.00000005e+00,
       -1.85833891e-08])

And you can show the "actual" data which is the event counts that we are fitting out model to.

In [37]:
m.expected_actualdata(bfit)

array([ 105675.6903921 ,  296032.53119449,  648237.46832765,
         61370.49624864,   89189.88320256, 1581806.72344248])

... or we can look at the full "data" list, which as we recall is both the event counts and the nuissance parameters which come into the constraint terms in the likelihood.

In [38]:
m.expected_data(bfit)

array([ 1.05675690e+05,  2.96032531e+05,  6.48237468e+05,  6.13704962e+04,
        8.91898832e+04,  1.58180672e+06,  5.94668452e-08, -3.34501004e-08,
       -1.85833891e-08,  9.99999990e-01, -1.67250502e-08,  0.00000000e+00,
       -1.67250502e-08, -1.48667113e-08, -5.57501673e-09, -2.41584058e-08,
        5.57501673e-08, -5.57501673e-09,  3.34501004e-08, -3.15917615e-08,
        5.76085062e-08])

Finally, we can take the difference between the fitted NPs and the initial values that we started with and that gives us the pull of that NP.

In [39]:
bfit-m.config.suggested_init()

array([ 5.94668452e-08,  0.00000000e+00, -1.01093637e-08, -3.34501004e-08,
       -1.67250502e-08, -1.48667113e-08, -5.57501673e-09,  5.57501673e-08,
        3.34501004e-08, -3.15917615e-08,  5.76085062e-08,  3.53084393e-08,
       -2.41584058e-08, -5.57501673e-09, -1.67250502e-08,  4.83168117e-08,
       -1.85833891e-08])

# Open Point
The open, and important point, is that the manner in which the error on a given NP is not well defined here in pyhf it seems.